# Earnings Call Project: Data Cleaning
<br>
CIS 831 Deep Learning – Term Project<br>
Kansas State University
<br><br>
James Chapman<br>
John Woods<br>
Nathan Diehl<br>
<br>

This notebook uses the transformer architecture from the [HTML paper](https://www.researchgate.net/publication/340385140_HTML_Hierarchical_Transformer-based_Multi-task_Learning_for_Volatility_Prediction). 

From the previous notebooks, we have features for both audio and text, of each sentence, of each meeting.<br>
- audio 
    - PRAAT (27 features)
- text
    - GLOVE (300 features)
    - ROBERTA (1024 features)
    - ROBERTA with averaging (1024 features)
    - FinLang investopedia (768 features) from huggingface sentencetransformers
    - BGE financial (1024 features) from huggingface sentencetransformers

This notebook performs 3 nested loops. Each audio/text pair, and for each N_DAYS, we train models with 17 different alpha values. The alpha value the lowest MSE, based on the validation set, is been used to train a model with Both training and validation sets. This model is finally tested on the test set.

These notebooks also give us a playground to work with the data, and test the models. Now we can insert sentiment detection, segmentation of the text, etc.

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn.functional as F
from torch import nn
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

np.random.seed(777)
torch.manual_seed(777)
torch.cuda.manual_seed_all(777)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
# static hyper-parameters from paper
max_norm = 1.0
num_epochs = 10
warmup_steps = 1000
batch_size=4
dropout=0.5
heads=2
depth=2

lr=0.000002 # 1/10 learning rate of paper

In [2]:
import shutil
import os

#fresh start

directory_path = r"C:\Users\James\OneDrive\Documents\GitHub\Earnings_call_project\runs"
if os.path.exists(directory_path):
    shutil.rmtree(directory_path)
    
path = "C:/Users/James/AppData/Local/Temp/.tensorboard-info/"
for filename in os.listdir(path):
    filepath = os.path.join(path, filename)
    print(filepath)
    os.remove(filepath)

# This is the hierarchical transformer from the HTML paper.

## This is taken directly from the GitHub account for HTML paper.
- a few adaptions are highlighted with ##############################################


- [GitHub HTML - util.py](https://github.com/YangLinyi/HTML-Hierarchical-Transformer-based-Multi-task-Learning-for-Volatility-Prediction/blob/master/Model/Sentence-Level-Transformer/transformers/util/util.py)
- [GitHub HTML - transformers_gpu.py](https://github.com/YangLinyi/HTML-Hierarchical-Transformer-based-Multi-task-Learning-for-Volatility-Prediction/blob/master/Model/Sentence-Level-Transformer/transformers/transformers_gpu.py)

In [4]:

# https://github.com/YangLinyi/HTML-Hierarchical-Transformer-based-Multi-task-Learning-for-Volatility-Prediction/blob/master/Model/Sentence-Level-Transformer/transformers/transformers_gpu.py
def mask_(matrices, maskval=0.0, mask_diagonal=True):
    """
    Masks out all values in the given batch of matrices where i <= j holds,
    i < j if mask_diagonal is false

    In place operation

    :param tns:
    :return:
    """

    b, h, w = matrices.size()

    indices = torch.triu_indices(h, w, offset=0 if mask_diagonal else 1)
    matrices[:, indices, indices[1]] = maskval

def contains_nan(tensor):
    return bool((tensor != tensor).sum() > 0)

# https://github.com/YangLinyi/HTML-Hierarchical-Transformer-based-Multi-task-Learning-for-Volatility-Prediction/blob/master/Model/Sentence-Level-Transformer/transformers/transformers_gpu.py
class SelfAttention(nn.Module):
    def __init__(self, emb, heads=8, mask=False):
        """
        :param emb:
        :param heads:
        :param mask:
        """

        super().__init__()

        self.emb = emb
        self.heads = heads
        self.mask = mask

        self.tokeys = nn.Linear(emb, emb * heads, bias=False)
        self.toqueries = nn.Linear(emb, emb * heads, bias=False)
        self.tovalues = nn.Linear(emb, emb * heads, bias=False)

        self.unifyheads = nn.Linear(heads * emb, emb)

    def forward(self, x):

        b, t, e = x.size()
        h = self.heads
        assert e == self.emb

        keys    = self.tokeys(x)   .view(b, t, h, e)
        queries = self.toqueries(x).view(b, t, h, e)
        values  = self.tovalues(x) .view(b, t, h, e)

        # compute scaled dot-product self-attention

        # - fold heads into the batch dimension
        keys = keys.transpose(1, 2).contiguous().view(b * h, t, e)
        queries = queries.transpose(1, 2).contiguous().view(b * h, t, e)
        values = values.transpose(1, 2).contiguous().view(b * h, t, e)

        queries = queries / (e ** (1/4))
        keys    = keys / (e ** (1/4))
        # - Instead of dividing the dot products by sqrt(e), we scale the keys and values.
        #   This should be more memory efficient

        # - get dot product of queries and keys, and scale
        dot = torch.bmm(queries, keys.transpose(1, 2))

        assert dot.size() == (b*h, t, t)

        if self.mask: # mask out the lower half of the dot matrix,including the diagonal
            mask_(dot, maskval=float('-inf'), mask_diagonal=False)

        dot = F.softmax(dot, dim=2) # dot now has row-wise self-attention probabilities

        ##############################################
        ##############################################
        # assert not util.contains_nan(dot[:, 1:, :]) # only the forst row may contain nan
        assert not contains_nan(dot[:, 1:, :]) # only the forst row may contain nan 
        ##############################################
        ##############################################
        
        if self.mask == 'first':
            dot = dot.clone()
            dot[:, :1, :] = 0.0
            # - The first row of the first attention matrix is entirely masked out, so the softmax operation results
            #   in a division by zero. We set this row to zero by hand to get rid of the NaNs

        # apply the self attention to the values
        out = torch.bmm(dot, values).view(b, h, t, e)

        # swap h, t back, unify heads
        out = out.transpose(1, 2).contiguous().view(b, t, h * e)

        return self.unifyheads(out)

class TransformerBlock(nn.Module):
    def __init__(self, emb, heads, mask, seq_length, ff_hidden_mult=4, dropout=0.0):
        super().__init__()

        self.attention = SelfAttention(emb, heads=heads, mask=mask)
        self.mask = mask

        self.norm1 = nn.LayerNorm(emb)
        self.norm2 = nn.LayerNorm(emb)

        self.ff = nn.Sequential(
            nn.Linear(emb, ff_hidden_mult * emb),
            nn.ReLU(),
            nn.Linear(ff_hidden_mult * emb, emb)
        )

        self.do = nn.Dropout(dropout)

    def forward(self, x):

        attended = self.attention(x)

        x = self.norm1(attended + x)

        x = self.do(x)

        fedforward = self.ff(x)

        x = self.norm2(fedforward + x)

        x = self.do(x)

        return x

class RTransformer(nn.Module):
    """
    Transformer for sequences Regression    
    
    """

    def __init__(self, emb, heads, depth, seq_length, num_tokens, num_classes, max_pool=True, dropout=0.0):
        """
        emb: Embedding dimension
        heads: nr. of attention heads
        depth: Number of transformer blocks
        seq_length: Expected maximum sequence length
        num_tokens: Number of tokens (usually words) in the vocabulary
        num_classes: Number of classes.
        max_pool: If true, use global max pooling in the last layer. If false, use global
                         average pooling.
        """
        super().__init__()

        self.num_tokens, self.max_pool = num_tokens, max_pool

        #self.token_embedding = nn.Embedding(embedding_dim=emb, num_embeddings=num_tokens)
        self.pos_embedding = nn.Embedding(embedding_dim=emb, num_embeddings=seq_length)

        tblocks = []
        for i in range(depth):
            tblocks.append(
                TransformerBlock(emb=emb, heads=heads, seq_length=seq_length, mask=False, dropout=dropout))

        self.tblocks = nn.Sequential(*tblocks)

        self.toprobs = nn.Linear(emb, num_classes)
        self.toprobs_b = nn.Linear(emb, num_classes)
        self.do = nn.Dropout(dropout)

    def forward(self, x):
        """
        :param x: A batch by sequence length integer tensor of token indices.
        :return: predicted log-probability vectors for each token based on the preceding tokens.
        """
        sentences_emb = x
        b, t, e = x.size()

        ##############################################
        ##############################################
        # swap d() for device
        # positions = self.pos_embedding(torch.arange(t, device=d()))[None, :, :].expand(b, t, e)
        positions = self.pos_embedding(torch.arange(t, device=device))[None, :, :].expand(b, t, e)
        ##############################################
        ##############################################
        #positions = self.pos_embedding(torch.arange(t))[None, :, :].expand(b, t, e)
        #positions = torch.tensor(positions, dtype=torch.float32)
        x = sentences_emb.cuda() + positions
        x = self.do(x)

        x = self.tblocks(x)

        x = x.max(dim=1)[0] if self.max_pool else x.mean(dim=1) # pool over the time dimension
        
        
        x_a = self.toprobs(x)
        x_b = self.toprobs_b(x)
        x_a = torch.squeeze(x_a)
        x_b = torch.squeeze(x_b)
        #print('x shape: ',x.shape)
        return x_a, x_b


# dataset builder and training loop

In [5]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, embeddings, labels, labels_b):
        self.embeddings = embeddings
        self.labels = labels
        self.labels_b = labels_b

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        emb = self.embeddings[idx]
        label = self.labels[idx]
        label_b = self.labels_b[idx]
        return emb, label, label_b 

def get_data(data_directory, n_days, year, no_validation_set, is_using_scaler, batch_size):
    # features
    train_features = np.load('data/{}/train_features{}.npy'.format(data_directory, year))
    val_features = np.load('data/{}/val_features{}.npy'.format(data_directory, year))
    test_features = np.load('data/{}/test_features{}.npy'.format(data_directory, year))
    # targets (n_days volatility)
    train_targets = np.load('data/{}/train_targets{}_{}.npy'.format(data_directory, year, n_days))
    val_targets = np.load('data/{}/val_targets{}_{}.npy'.format(data_directory, year, n_days))
    test_targets = np.load('data/{}/test_targets{}_{}.npy'.format(data_directory, year, n_days))
    # secondary targets (log percent change of day n)
    train_secondary_targets = np.load('data/{}/train_secondary_targets{}_{}.npy'.format(data_directory, year, n_days))
    val_secondary_targets = np.load('data/{}/val_secondary_targets{}_{}.npy'.format(data_directory, year, n_days))
    test_secondary_targets = np.load('data/{}/test_secondary_targets{}_{}.npy'.format(data_directory, year, n_days))

    # after hyperparameters are tuned on the validation set
    # add validation set to training set
    if no_validation_set:
        train_features = np.concatenate((train_features, val_features), axis=0)
        train_targets = np.concatenate((train_targets, val_targets), axis=0)
        train_secondary_targets = np.concatenate((train_secondary_targets, val_secondary_targets), axis=0)

    if is_using_scaler:
        # Scaling features
        feature_scaler = StandardScaler()
        train_features = feature_scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
        val_features = feature_scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
        test_features = feature_scaler.transform(test_features.reshape(-1, test_features.shape[-1])).reshape(test_features.shape)
        # Scaling primary targets
        target_scaler = StandardScaler()
        train_targets = target_scaler.fit_transform(train_targets.reshape(-1, 1)).reshape(train_targets.shape)
        val_targets = target_scaler.transform(val_targets.reshape(-1, 1)).reshape(val_targets.shape)
        test_targets = target_scaler.transform(test_targets.reshape(-1, 1)).reshape(test_targets.shape)
        # Scaling secondary targets
        secondary_target_scaler = StandardScaler()
        train_secondary_targets = secondary_target_scaler.fit_transform(train_secondary_targets.reshape(-1, 1)).reshape(train_secondary_targets.shape)
        val_secondary_targets = secondary_target_scaler.transform(val_secondary_targets.reshape(-1, 1)).reshape(val_secondary_targets.shape)
        test_secondary_targets = secondary_target_scaler.transform(test_secondary_targets.reshape(-1, 1)).reshape(test_secondary_targets.shape)
    # Dataset & DataLoader
    training_set = Dataset(train_features, train_targets, train_secondary_targets) 
    val_set = Dataset(val_features, val_targets, val_secondary_targets)
    test_set = Dataset(test_features, test_targets, test_secondary_targets)
    trainloader = torch.utils.data.DataLoader(training_set, batch_size=batch_size, shuffle=True, num_workers=0)
    valloader = torch.utils.data.DataLoader(val_set, batch_size=len(val_set), shuffle=False, num_workers=0)
    testloader = torch.utils.data.DataLoader(test_set, batch_size=len(test_set), shuffle=False, num_workers=0)

    print(train_features.shape, train_targets.shape, train_secondary_targets.shape) 
    print(val_features.shape, val_targets.shape, val_secondary_targets.shape) 
    print(test_features.shape, test_targets.shape, test_secondary_targets.shape) 

    # train_features.shape[2] is the number of features, need to instantiate the model
    # need target scaler to inverse transform
    return trainloader, valloader, testloader, train_features.shape[2], target_scaler 


In [6]:
# this training loop is actually reused for training, validation, and testing
def  training_loop(trainloader, loader, emb_dimensions, heads, depth,dropout, warmup_steps, 
                   batch_size, num_epochs, alphas, max_norm, lr, run_name, target_scaler):
    alphas_results = []
    for alpha in alphas:
        model = RTransformer(emb=emb_dimensions, heads=heads, depth=depth, seq_length=523, 
                            num_tokens=4000, num_classes=1, max_pool=False, dropout=dropout).to(device)
        opt = torch.optim.Adam(model.parameters(), lr=lr)
        # Linear LR warmup for the first #warmup_steps training examples
        scheduler = LambdaLR(opt, lr_lambda=lambda step: min(1.0, step / (warmup_steps/batch_size)))

        seen = 0
        val_loss_a_history =  [] # keep track of the loss for the current Alpha value
        writer = SummaryWriter(log_dir= f'runs/{run_name}_{alpha}')
        progress_bar = tqdm(range(num_epochs), desc="Training Progress", unit="epoch")
        for epoch in progress_bar:
            model.train()
            train_loss_total = 0
            for i, (inputs, labels, labels_b) in enumerate(trainloader): #training data
                seen += inputs.size(0)
                inputs = inputs.to(device, dtype=torch.float32)
                labels = labels.to(device, dtype=torch.float32)
                labels_b = labels_b.to(device, dtype=torch.float32)
                out_a, out_b = model(inputs)
                out_a = out_a.view(-1)
                out_b = out_b.view(-1)

                # Compute the combined loss (multitask)
                loss_a = F.mse_loss(out_a, labels)
                loss_b = F.mse_loss(out_b, labels_b)
                loss = alpha * loss_a + (1 - alpha) * loss_b # Alpha parameter
            
                opt.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_norm)
                opt.step()
                if seen < warmup_steps:
                    scheduler.step()

                # we only care about the primary target
                # although we are training with the scaled loss, reporting the volatility must be unscaled
                out_a_unscaled = target_scaler.inverse_transform(out_a.detach().cpu().numpy().reshape(-1, 1)).reshape(out_a.shape)
                labels_unscaled = target_scaler.inverse_transform(labels.detach().cpu().numpy().reshape(-1, 1)).reshape(labels.shape)
                loss_a_unscaled = F.mse_loss(torch.tensor(out_a_unscaled, device=device), torch.tensor(labels_unscaled, device=device))
                train_loss_total += loss_a_unscaled.item() * inputs.size(0) # MSE of volatility training (multiplied by current batch size)

            # Epoch average of unscaled training loss
            train_loss_avg = train_loss_total / len(trainloader.dataset)
            
            # Epoch Evaluation
            model.eval()
            val_loss_a_total = 0.0
            with torch.no_grad():
                for i, (inputs, labels, labels_b) in enumerate(loader): #validation or testing
                    inputs = inputs.to(device, dtype=torch.float32)
                    labels = labels.to(device, dtype=torch.float32)
                    labels_b = labels_b.to(device, dtype=torch.float32)

                    out_a, out_b = model(inputs)
                    out_a = out_a.view(-1)
                    
                    out_a_unscaled = target_scaler.inverse_transform(out_a.detach().cpu().numpy().reshape(-1, 1)).reshape(out_a.shape)
                    labels_unscaled = target_scaler.inverse_transform(labels.detach().cpu().numpy().reshape(-1, 1)).reshape(labels.shape)
                    loss_a_unscaled = F.mse_loss(torch.tensor(out_a_unscaled, device=device), torch.tensor(labels_unscaled, device=device))
                    val_loss_a_total += loss_a_unscaled.item() * inputs.size(0) # MSE of volatility

                val_loss_a_avg = val_loss_a_total / len(loader.dataset) # MSE of volatility of Val/test!!
            # Epoch finished
            writer.add_scalar('Train Loss/train', train_loss_avg, epoch)
            writer.add_scalar('Loss A/val', val_loss_a_avg, epoch)
            # Update progress bar description and postfix
            progress_bar.set_description(f"Epoch {epoch+1}/{num_epochs}")
            progress_bar.set_postfix({
                "Train Loss": f"{train_loss_avg:.3f}",
                "Test Loss A": f"{val_loss_a_avg:.3f}"
            })
            # for alpha optimization, collect each Epoch's average loss
            val_loss_a_history.append(val_loss_a_avg)
        # all Epochs finished for this alpha value
        min_loss = round(min(val_loss_a_history), 3)
        alphas_results.append((alpha, min_loss))
        writer.close()
    print(alphas_results)
    best_alpha, lowest_val_loss = min(alphas_results, key=lambda x: x[1])
    return best_alpha, lowest_val_loss # if testING, this is the MSE  of the test set

In [7]:

def train_val_test(data_directory, n_days, year):
    run_name = data_directory + year + '_' + n_days 
    # using training set and validation set, determine optimum hyper-parameters (just alpha)
    no_validation_set = False
    trainloader,valloader,testloader,emb_dimensions, target_scaler = get_data(data_directory, n_days, year, no_validation_set, is_using_scaler, batch_size)
    best_alpha, _ = training_loop(trainloader, valloader, emb_dimensions, heads, depth,dropout, warmup_steps, 
                            batch_size, num_epochs, alphas, max_norm, lr, run_name, target_scaler)
    # using optimum alpha, retrain with all training/validation sets and test on testset
    test_run_name = run_name + '_test'
    no_validation_set = True
    trainloader,valloader,testloader,emb_dimensions, target_scaler = get_data(data_directory, n_days, year, no_validation_set, is_using_scaler, batch_size)
    _, MSE_testset = training_loop(trainloader, testloader, emb_dimensions, heads, depth, dropout, warmup_steps, 
                        batch_size, num_epochs, [best_alpha], max_norm, lr, test_run_name, target_scaler)

    print('----------------------------------------')
    print(f'run_name-----------{run_name}')
    print(f'best_alpha---------{best_alpha}')
    print(f'MSE_testset--------{MSE_testset}')
    print('----------------------------------------')
    return best_alpha, MSE_testset

# Train & Test original dataset

In [ ]:

is_using_scaler = True # whether or not to use standardscaler
directories = ['Roberta', 'Roberta2', 'investopedia', 'bge', 'bge_base', 'glove']#
all_n_days = ['3', '7', '15', '30']
alphas = [0.1, 0.3, 0.5, 0.7, 0.9]

final_results = []
for data_directory in directories: # different feature engineering features
    for n_days in all_n_days: 
        year = ''
        best_alpha, MSE_testset = train_val_test(data_directory, n_days, year)
        final_results.append([data_directory, n_days, best_alpha, MSE_testset])

(391, 523, 1051) (391,) (391,)
(54, 523, 1051) (54,) (54,)
(115, 523, 1051) (115,) (115,)


Epoch 10/10: 100%|██████████| 10/10 [00:24<00:00,  2.41s/epoch, Train Loss=0.751, Test Loss A=0.847]


[(0.1, 0.48), (0.3, 0.535), (0.5, 0.533), (0.7, 0.527), (0.9, 0.525)]
(445, 523, 1051) (445,) (445,)
(54, 523, 1051) (54,) (54,)
(115, 523, 1051) (115,) (115,)


Epoch 10/10: 100%|██████████| 10/10 [00:28<00:00,  2.88s/epoch, Train Loss=0.721, Test Loss A=0.699]


[(0.1, 0.687)]
----------------------------------------
run_name-----------Roberta_3
best_alpha---------0.1
MSE_testset--------0.687
----------------------------------------
(391, 523, 1051) (391,) (391,)
(54, 523, 1051) (54,) (54,)
(115, 523, 1051) (115,) (115,)


...

[(0.5, 0.262)]
----------------------------------------
run_name-----------glove_15
best_alpha---------0.5
MSE_testset--------0.262
----------------------------------------
(391, 523, 327) (391,) (391,)
(54, 523, 327) (54,) (54,)
(115, 523, 327) (115,) (115,)


Epoch 10/10: 100%|██████████| 10/10 [00:06<00:00,  1.65epoch/s, Train Loss=0.189, Test Loss A=0.110]


[(0.1, 0.116), (0.3, 0.111), (0.5, 0.114), (0.7, 0.114), (0.9, 0.11)]
(445, 523, 327) (445,) (445,)
(54, 523, 327) (54,) (54,)
(115, 523, 327) (115,) (115,)


Epoch 10/10: 100%|██████████| 10/10 [00:07<00:00,  1.32epoch/s, Train Loss=0.180, Test Loss A=0.214]

[(0.9, 0.2)]
----------------------------------------
run_name-----------glove_30
best_alpha---------0.9
MSE_testset--------0.2
----------------------------------------


In [ ]:
final_results = pd.DataFrame(final_results, columns=['Embedding Variant', 'n_days', 'best_alpha', 'MSE_testset'])
final_results.to_csv('data/final_results.csv', index=False)
final_results

,Embedding Variant,n_days,best_alpha,MSE_testset
0,Roberta,3,0.1,0.687
1,Roberta,7,0.5,0.346
2,Roberta,15,0.5,0.238
3,Roberta,30,0.1,0.189
4,Roberta2,3,0.1,0.678
5,Roberta2,7,0.3,0.316
6,Roberta2,15,0.9,0.244
7,Roberta2,30,0.5,0.185
8,investopedia,3,0.1,0.726
9,investopedia,7,0.3,0.337


In [ ]:
final_results = final_results.drop(columns=['best_alpha'])
final_results = final_results.pivot(index='Embedding Variant', columns='n_days', values='MSE_testset')
final_results

n_days,3,7,15,30
Embedding Variant,,,,
Roberta,0.687,0.346,0.238,0.189
Roberta2,0.678,0.316,0.244,0.185
bge,0.677,0.351,0.239,0.184
bge_base,0.677,0.331,0.248,0.187
glove,0.748,0.383,0.262,0.200
investopedia,0.726,0.337,0.243,0.193


# Train & Test MAEC dataset

In [ ]:
directories = ['MAEC_Roberta', 'MAEC_Roberta2', 'MAEC_investopedia', 'MAEC_bge', 'MAEC_bge_base', 'MAEC_glove']#
years = ['_2015', '_2016', '_2017']

MAEC_final_results = []
for data_directory in directories: # different feature engineering features
    for n_days in all_n_days: 
        for year in years:
            best_alpha, MSE_testset = train_val_test(data_directory, n_days, year)
            MAEC_final_results.append([data_directory, n_days, year, best_alpha, MSE_testset])

(485, 500, 1051) (485,) (485,)
(67, 500, 1051) (67,) (67,)
(146, 500, 1051) (146,) (146,)


Epoch 10/10: 100%|██████████| 10/10 [00:28<00:00,  2.85s/epoch, Train Loss=0.754, Test Loss A=1.071]


[(0.1, 0.893), (0.3, 0.943), (0.5, 0.924), (0.7, 0.924), (0.9, 0.908)]
(552, 500, 1051) (552,) (552,)
(67, 500, 1051) (67,) (67,)
(146, 500, 1051) (146,) (146,)


Epoch 10/10: 100%|██████████| 10/10 [00:33<00:00,  3.38s/epoch, Train Loss=0.782, Test Loss A=0.718]


[(0.1, 0.718)]
----------------------------------------
run_name-----------MAEC_Roberta_2015_3
best_alpha---------0.1
MSE_testset--------0.718
----------------------------------------
(900, 500, 1051) (900,) (900,)
(142, 500, 1051) (142,) (142,)
(266, 500, 1051) (266,) (266,)


...

[(0.9, 0.128)]
----------------------------------------
run_name-----------MAEC_glove_2016_30
best_alpha---------0.9
MSE_testset--------0.128
----------------------------------------
(827, 500, 327) (827,) (827,)
(128, 500, 327) (128,) (128,)
(238, 500, 327) (238,) (238,)


Epoch 10/10: 100%|██████████| 10/10 [00:12<00:00,  1.28s/epoch, Train Loss=0.230, Test Loss A=0.262]


[(0.1, 0.284), (0.3, 0.24), (0.5, 0.266), (0.7, 0.238), (0.9, 0.257)]
(955, 500, 327) (955,) (955,)
(128, 500, 327) (128,) (128,)
(238, 500, 327) (238,) (238,)


Epoch 10/10: 100%|██████████| 10/10 [00:15<00:00,  1.57s/epoch, Train Loss=0.231, Test Loss A=0.293]

[(0.7, 0.276)]
----------------------------------------
run_name-----------MAEC_glove_2017_30
best_alpha---------0.7
MSE_testset--------0.276
----------------------------------------


In [ ]:
MAEC_final_results = pd.DataFrame(MAEC_final_results, columns=['Embedding Variant', 'n_days', 'year', 'best_alpha', 'MSE_testset'])
MAEC_final_results.to_csv('data/MAEC_final_results.csv', index=False)
MAEC_final_results

,Embedding Variant,n_days,year,best_alpha,MSE_testset
0,MAEC_Roberta,3,_2015,0.1,0.718
1,MAEC_Roberta,3,_2016,0.7,0.686
2,MAEC_Roberta,3,_2017,0.5,0.868
3,MAEC_Roberta,7,_2015,0.3,0.377
4,MAEC_Roberta,7,_2016,0.9,0.338
...,...,...,...,...,...
67,MAEC_glove,15,_2016,0.1,0.193
68,MAEC_glove,15,_2017,0.7,0.333
69,MAEC_glove,30,_2015,0.5,0.204
70,MAEC_glove,30,_2016,0.9,0.128


# 2015

In [ ]:
MAEC_final_results_2015 = MAEC_final_results[MAEC_final_results['year']=='_2015']
MAEC_final_results_2015 = MAEC_final_results_2015.drop(['year'], axis=1)
MAEC_final_results_2015 = MAEC_final_results_2015.pivot(index='Embedding Variant', columns='n_days', values='MSE_testset')
MAEC_final_results_2015 = MAEC_final_results_2015[['3','7','15','30']]
MAEC_final_results_2015

n_days,3,7,15,30
Embedding Variant,,,,
MAEC_Roberta,0.718,0.377,0.252,0.189
MAEC_Roberta2,0.675,0.359,0.234,0.179
MAEC_bge,0.670,0.364,0.240,0.178
MAEC_bge_base,0.697,0.364,0.235,0.186
MAEC_glove,0.721,0.398,0.263,0.204
MAEC_investopedia,0.672,0.356,0.237,0.189


# 2016

In [ ]:
MAEC_final_results_2016 = MAEC_final_results[MAEC_final_results['year']=='_2016']
MAEC_final_results_2016 = MAEC_final_results_2016.drop(['year'], axis=1)
MAEC_final_results_2016 = MAEC_final_results_2016.pivot(index='Embedding Variant', columns='n_days', values='MSE_testset')
MAEC_final_results_2016 = MAEC_final_results_2016[['3','7','15','30']]
MAEC_final_results_2016

n_days,3,7,15,30
Embedding Variant,,,,
MAEC_Roberta,0.686,0.338,0.185,0.131
MAEC_Roberta2,0.678,0.318,0.197,0.131
MAEC_bge,0.671,0.320,0.188,0.129
MAEC_bge_base,0.677,0.305,0.193,0.130
MAEC_glove,0.700,0.345,0.193,0.128
MAEC_investopedia,0.690,0.312,0.185,0.130


# 2017

In [ ]:
MAEC_final_results_2017 = MAEC_final_results[MAEC_final_results['year']=='_2017']
MAEC_final_results_2017 = MAEC_final_results_2017.drop(['year'], axis=1)
MAEC_final_results_2017 = MAEC_final_results_2017.pivot(index='Embedding Variant', columns='n_days', values='MSE_testset')
MAEC_final_results_2017 = MAEC_final_results_2017[['3','7','15','30']]
MAEC_final_results_2017

n_days,3,7,15,30
Embedding Variant,,,,
MAEC_Roberta,0.868,0.399,0.326,0.267
MAEC_Roberta2,0.829,0.399,0.263,0.207
MAEC_bge,0.739,0.413,0.307,0.231
MAEC_bge_base,0.737,0.382,0.295,0.217
MAEC_glove,0.781,0.428,0.333,0.276
MAEC_investopedia,0.738,0.400,0.290,0.233


In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs 

In [ ]:
:)

SyntaxError: unmatched ')' (1896645534.py, line 1)